### Extracción de datos a través de la api "Datos Abiertos Comunidad de Madrid"
https://datos.comunidad.madrid/catalogo/api/action/datastore_search?id=f22c3f43-c5d0-41a4-96dc-719214d56968

In [1]:
import urllib
import urllib3
import certifi
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

In [3]:
#paso2:acceder a la url para buscar los datos 
url="https://datos.comunidad.madrid/catalogo/api/action/datastore_search?id=f22c3f43-c5d0-41a4-96dc-719214d56968"
r = http.request('GET', url)
r.status

200

In [4]:
data = json.loads(r.data.decode('utf-8'))
data

{'help': 'https://datos.comunidad.madrid/catalogo/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'resource_id': 'f22c3f43-c5d0-41a4-96dc-719214d56968',
  'fields': [{'type': 'int', 'id': '_id'},
   {'info': {'notes': 'Municipio / Distrito Municipal: Literal de municipio, o Literal de distrito para el municipio de Madrid',
     'type_override': '',
     'label': ''},
    'type': 'text',
    'id': 'municipio_distrito'},
   {'info': {'notes': 'Fecha de generación del fichero de datos para la elaboración del informe',
     'type_override': '',
     'label': ''},
    'type': 'timestamp',
    'id': 'fecha_informe'},
   {'info': {'notes': 'Número de casos confirmados con infección activa de cada ámbito territorial con fecha de consulta médica o diagnóstico en las dos últimas semanas epidemiológicas previas a la fecha del informe.  \r\nA partir del 13 de octubre de 2020 este campo aparece con valor nulo.',
     'type_override': '',
     'l

In [5]:
type(data)

dict

In [6]:
## con esto obtengo el total de registros que viene en la consulta
total=data['result']['total']
total

7136

In [7]:
#vuelvo a contruir la url, ya que me dí cuenta que la anterior sólo me trae 100 registros
url="https://datos.comunidad.madrid/catalogo/api/action/datastore_search?id=f22c3f43-c5d0-41a4-96dc-719214d56968"
limit='&limit='+str(total)
url_final=url+limit
r = http.request('GET', url_final)
r.status

200

In [8]:
data = json.loads(r.data.decode('utf-8'))
data

{'help': 'https://datos.comunidad.madrid/catalogo/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'resource_id': 'f22c3f43-c5d0-41a4-96dc-719214d56968',
  'fields': [{'type': 'int', 'id': '_id'},
   {'info': {'notes': 'Municipio / Distrito Municipal: Literal de municipio, o Literal de distrito para el municipio de Madrid',
     'type_override': '',
     'label': ''},
    'type': 'text',
    'id': 'municipio_distrito'},
   {'info': {'notes': 'Fecha de generación del fichero de datos para la elaboración del informe',
     'type_override': '',
     'label': ''},
    'type': 'timestamp',
    'id': 'fecha_informe'},
   {'info': {'notes': 'Número de casos confirmados con infección activa de cada ámbito territorial con fecha de consulta médica o diagnóstico en las dos últimas semanas epidemiológicas previas a la fecha del informe.  \r\nA partir del 13 de octubre de 2020 este campo aparece con valor nulo.',
     'type_override': '',
     'l

In [9]:
type(data)

dict

In [10]:
for i in data:
    print(i)

help
success
result


In [11]:
df=data['result']['records']

In [12]:
df=pd.DataFrame(df)
df.head()

,_id,municipio_distrito,fecha_informe,casos_confirmados_activos_ultimos_14dias,tasa_incidencia_acumulada_activos_ultimos_14dias,casos_confirmados_ultimos_14dias,tasa_incidencia_acumulada_ultimos_14dias,casos_confirmados_totales,tasa_incidencia_acumulada_total,codigo_geometria
0,1,Madrid-Retiro,2021-01-26T11:08:00,,,1122.0,"940,35",8395.0,"7035,88",79603.0
1,2,Madrid-Salamanca,2021-01-26T11:08:00,,,1411.0,"965,75",10684.0,"7312,6",79604.0
2,3,Madrid-Centro,2021-01-26T11:08:00,,,1235.0,"915,85",10249.0,"7600,41",79601.0
3,4,Madrid-Arganzuela,2021-01-26T11:08:00,,,1364.0,"886,85",10963.0,"7127,95",79602.0
4,5,Madrid-Chamartín,2021-01-26T11:08:00,,,1439.0,"986,64",10344.0,"7092,27",79605.0


In [ ]:
#confirmo que ahora me trae el total de registros
len(df)

In [13]:
df2=df.drop(['_id','casos_confirmados_activos_ultimos_14dias', 'tasa_incidencia_acumulada_activos_ultimos_14dias','codigo_geometria',
             'tasa_incidencia_acumulada_ultimos_14dias','tasa_incidencia_acumulada_total'], axis=1)
df2.head()

,municipio_distrito,fecha_informe,casos_confirmados_ultimos_14dias,casos_confirmados_totales
0,Madrid-Retiro,2021-01-26T11:08:00,1122.0,8395.0
1,Madrid-Salamanca,2021-01-26T11:08:00,1411.0,10684.0
2,Madrid-Centro,2021-01-26T11:08:00,1235.0,10249.0
3,Madrid-Arganzuela,2021-01-26T11:08:00,1364.0,10963.0
4,Madrid-Chamartín,2021-01-26T11:08:00,1439.0,10344.0


In [14]:
df2.describe()

,casos_confirmados_ultimos_14dias,casos_confirmados_totales
count,3876.000000,6262.000000
mean,185.067079,1267.618333
std,335.217204,2736.564798
min,5.000000,5.000000
25%,15.000000,37.000000
50%,38.000000,151.000000
75%,189.000000,880.750000
max,2988.000000,24039.000000


In [15]:
df2.dtypes

municipio_distrito                   object
fecha_informe                        object
casos_confirmados_ultimos_14dias    float64
casos_confirmados_totales           float64
dtype: object

In [16]:
df2.to_csv("casoscovidmadrid.csv",index=False,sep=';',decimal=',')

## Buscar geolocazación de los distritos de Madrid

In [18]:
# pip install geopy

In [17]:
from geopy.geocoders import Nominatim
import time

In [19]:
## creando un objeto nominatim
geo=Nominatim(user_agent='Marchelo')

In [20]:
# recorrer la lista para solicitar su localizacion 
#usamos timeout para no saturar la api

In [21]:
distritos=df2['municipio_distrito'].unique().tolist()

In [22]:
distritos

['Madrid-Retiro',
 'Madrid-Salamanca',
 'Madrid-Centro',
 'Madrid-Arganzuela',
 'Madrid-Chamartín',
 'Madrid-Tetuán',
 'Madrid-Chamberí',
 'Madrid-Fuencarral-El Pardo',
 'Madrid-Moncloa-Aravaca',
 'Madrid-Latina',
 'Madrid-Carabanchel',
 'Madrid-Usera',
 'Madrid-Puente de Vallecas',
 'Madrid-San Blas - Canillejas',
 'Madrid-Barajas',
 'Madrid-Moratalaz',
 'Madrid-Ciudad Lineal',
 'Madrid-Hortaleza',
 'Madrid-Villaverde',
 'Madrid-Villa de Vallecas',
 'Madrid-Vicálvaro',
 'Pozuelo de Alarcón',
 'Alcorcón',
 'Navalcarnero',
 'San Agustín del Guadalix',
 'Las Rozas de Madrid',
 'Moraleja de Enmedio',
 'Valdaracete',
 'Pozuelo del Rey',
 'Puebla de la Sierra',
 'Perales de Tajuña',
 'Valdemorillo',
 'Navalafuente',
 'Velilla de San Antonio',
 'Villanueva de la Cañada',
 'Villanueva del Pardillo',
 'Nuevo Baztán',
 'Villamantilla',
 'Villamanta',
 'Cabanillas de la Sierra',
 'Cenicientos',
 'Puentes Viejas',
 'Aranjuez',
 'Cercedilla',
 'Santorcaz',
 'Coslada',
 'Morata de Tajuña',
 'Robreg

In [23]:
latitud=[]
longitud=[]
for i in distritos:
    address=geo.geocode(i,timeout=1)
    latitud.append(address.latitude)
    longitud.append(address.longitude)

In [24]:
coordenadas=pd.DataFrame()
type(coordenadas)

pandas.core.frame.DataFrame

In [25]:
coordenadas['Distrito']=distritos

In [26]:
coordenadas['Latitud']=latitud

In [27]:
coordenadas['Longitud']=longitud

In [28]:
latitud

[40.4111495,
 40.4270451,
 40.417652700000005,
 40.39806845,
 40.4589872,
 40.4605781,
 40.43624735,
 40.55634555,
 40.43949485,
 40.4035317,
 40.3742112,
 40.383894,
 40.3835532,
 40.428919050000005,
 40.4733176,
 40.4059332,
 40.4484305,
 40.4725491,
 40.3456104,
 40.3739576,
 40.3965841,
 40.4346528,
 40.3492998,
 40.2877169,
 40.679051,
 40.4933095,
 40.2615532,
 40.2104156,
 40.3664055,
 41.0121247,
 40.2325718,
 40.5016772,
 40.8235296,
 40.367166,
 40.4472106,
 40.4949232,
 40.3659255,
 40.3421033,
 40.2990939,
 40.8272445,
 40.2631894,
 40.968658149999996,
 40.03221,
 40.7412931,
 40.4715789,
 40.423802,
 40.2294318,
 41.1064517,
 41.0059166,
 40.7987856,
 40.7287926,
 41.0593529,
 40.5922684,
 40.9261949,
 40.5110297,
 40.3863666,
 40.3695225,
 40.8586659,
 40.7163374,
 40.282476,
 40.9216007,
 40.240985,
 40.9188552,
 40.9756034,
 40.5012957,
 40.2315094,
 40.3605272,
 40.5774992,
 40.2136607,
 40.96551504999999,
 40.3215212,
 40.256626,
 40.7460256,
 40.379164,
 40.1580087,


In [30]:
coordenadas.head()

,Distrito,Latitud,Longitud
0,Madrid-Retiro,40.411150,-3.676057
1,Madrid-Salamanca,40.427045,-3.680602
2,Madrid-Centro,40.417653,-3.707914
3,Madrid-Arganzuela,40.398068,-3.693734
4,Madrid-Chamartín,40.458987,-3.676129


In [31]:
coordenadas.dtypes

Distrito     object
Latitud     float64
Longitud    float64
dtype: object

In [32]:
coordenadas.to_csv("CoordenadasMadrid.csv",index=False,sep=';',decimal=',')